# Robótica Móvel - Trabalho Prático 1 (Exercícios 5-6)
---
**Nome Completo:** Daniel Terra Gomes

**Matrícula:** 2025702870 

**Programa:** Mestrando do PPGCC  
**Departamento:** Departamento de Ciência da Computação  
**Instituição:** ICEx-UFMG  
**E-mail:** danielterragomes@ufmg.br

**Data:** 11 de setembro de 2025

---

Este notebook implementa os exercícios 5-6 do TP1, focando em:
- Integração de sensores laser (Hokuyo)
- Transformação de dados de sensores para frame global
- Navegação autônoma básica
- Mapeamento incremental com dados de laser

**Atenção:** Use a cena `T1-ex5.ttt` que contém o robô Pioneer P3DX com sensor laser.

## Configuração Inicial e Importações

In [ ]:
# Imports necessários
import numpy as np
import matplotlib.pyplot as plt
import time
from typing import Dict, List, Tuple, Optional

# Import das utilidades criadas para o TP1
from robotics_utils import (
    CoppeliaSimConnector,
    HokuyoSensorSim,
    RobotController,
    SceneAnalyzer,
    create_homogeneous_matrix,
    invert_homogeneous_matrix,
    draw_laser_data,
    plot_robot_path,
    transform_laser_to_global,
    simple_obstacle_avoidance,
    DEFAULT_OBJECT_MAPPING_EX5_6,
    setup_simulation,
    wait_for_user_input
)

print("Módulos importados com sucesso!")
print("Certifique-se de que o CoppeliaSim está rodando com a cena T1-ex5.ttt aberta.")
print("Esta cena deve conter o robô Pioneer P3DX com sensor laser Hokuyo.")

## Conexão com CoppeliaSim e Configuração da Simulação

In [ ]:
# Conectar ao CoppeliaSim
print("Conectando ao CoppeliaSim...")
connector = CoppeliaSimConnector()

if not connector.connect():
    raise RuntimeError("Falha na conexão com CoppeliaSim. Verifique se o simulador está rodando.")

# Configurar simulação em modo stepped
connector.sim.setStepping(True)
print("Modo de simulação stepped ativado.")

# Parar simulação se estiver rodando
initial_sim_state = connector.sim.getSimulationState()
if initial_sim_state != 0:
    connector.sim.stopSimulation()
    time.sleep(1)
    print("Simulação anterior interrompida.")

print("Sistema pronto para os exercícios 5-6.")

## Exercício 5: Integração do Sensor Laser

Neste exercício, substituímos o robô inicial pelo Pioneer P3DX equipado com sensor laser Hokuyo. O objetivo é:

1. Definir as matrizes de transformação **T_L_R** (laser → robô) e **T_W_R** (robô → mundo)
2. Modificar o script para plotar pontos do laser no referencial global
3. Testar em diferentes posições do robô

**Fundamentação teórica:**
- Cadeia de transformações: Laser → Robô → Mundo
- Transformação completa: **T_W_L = T_W_R @ T_R_L**
- Pontos globais: **P_world = T_W_L @ P_laser**

In [ ]:
# Inicializar componentes do robô
print("Inicializando componentes do robô...")

try:
    # Controlador do robô Pioneer P3DX
    robot_controller = RobotController(
        sim=connector.sim,
        robot_name='PioneerP3DX',
        wheel_separation=0.381,  # 38.1 cm
        wheel_radius=0.0975      # 9.75 cm
    )
    print("✓ Controlador do robô inicializado")

    # Sensor laser Hokuyo
    hokuyo_sensor = HokuyoSensorSim(
        sim=connector.sim,
        base_name='/PioneerP3DX/fastHokuyo',
        is_range_data=True  # Retorna dados como [ângulo, distância]
    )
    print("✓ Sensor Hokuyo inicializado")

except Exception as e:
    print(f"❌ Erro na inicialização: {e}")
    print("Verifique se a cena T1-ex5.ttt está carregada com o Pioneer P3DX.")
    raise

### Definição das Matrizes de Transformação

In [ ]:
def get_transformation_matrices(robot_controller, connector):
    """
    Calcula as matrizes de transformação necessárias.

    Returns:
        tuple: (T_W_R, T_R_L) - Transformações mundo->robô e robô->laser
    """
    # Obter pose atual do robô no mundo
    robot_pose = robot_controller.get_pose()
    if not robot_pose:
        raise RuntimeError("Não foi possível obter pose do robô")

    position, orientation = robot_pose

    # T_W_R: Transformação do mundo para o robô
    T_W_R = create_homogeneous_matrix(position, orientation)

    # T_R_L: Transformação do robô para o laser
    # O laser está montado no robô, tipicamente no centro ou ligeiramente à frente
    # Para o Pioneer P3DX com fastHokuyo, assumimos que o laser está no centro do robô
    laser_offset = np.array([0.0, 0.0, 0.1])  # 10cm acima do robô
    laser_orientation = np.array([0.0, 0.0, 0.0])  # Mesma orientação do robô

    T_R_L = create_homogeneous_matrix(laser_offset, laser_orientation)

    return T_W_R, T_R_L

def test_laser_in_position(robot_controller, hokuyo_sensor, position_name):
    """
    Testa o sensor laser em uma posição específica do robô.
    """
    print(f"\n=== Teste do Laser - {position_name} ===")

    # Obter dados do laser
    laser_data = hokuyo_sensor.getSensorData()
    print(f"Laser detectou {len(laser_data)} pontos")

    if len(laser_data) == 0:
        print("⚠️ Nenhum dado de laser detectado")
        return None

    # Obter matrizes de transformação
    T_W_R, T_R_L = get_transformation_matrices(robot_controller, connector)

    # Mostrar informações da pose do robô
    robot_pose = robot_controller.get_pose()
    if robot_pose:
        pos, ori = robot_pose
        print(f"Pose do robô: [{pos[0]:.3f}, {pos[1]:.3f}, {pos[2]:.3f}] m")
        print(f"Orientação: [{np.rad2deg(ori[0]):.1f}°, {np.rad2deg(ori[1]):.1f}°, {np.rad2deg(ori[2]):.1f}°]")

    # Plot 1: Dados do laser no frame local
    print("Plotando dados do laser no frame local...")
    draw_laser_data(laser_data, title=f"Laser Local Frame - {position_name}")

    # Transformar pontos para frame global
    global_points = transform_laser_to_global(laser_data, robot_pose)
    print(f"Transformados {len(global_points)} pontos para frame global")

    # Plot 2: Dados do laser no frame global
    if len(global_points) > 0:
        plt.figure(figsize=(10, 8))
        plt.scatter(global_points[:, 0], global_points[:, 1], c='red', s=2, alpha=0.7, label='Pontos Laser')

        # Plotar posição do robô
        plt.plot(pos[0], pos[1], 'bo', markersize=8, label='Robô')

        # Plotar orientação do robô
        robot_direction = np.array([np.cos(ori[2]), np.sin(ori[2])]) * 0.5
        plt.arrow(pos[0], pos[1], robot_direction[0], robot_direction[1],
                 head_width=0.1, head_length=0.1, fc='blue', ec='blue')

        plt.axis('equal')
        plt.grid(True)
        plt.xlabel('X (metros)')
        plt.ylabel('Y (metros)')
        plt.title(f'Dados do Laser no Frame Global - {position_name}')
        plt.legend()
        plt.show()

    return global_points, T_W_R, T_R_L

print("Funções de transformação definidas.")

### Teste Inicial do Laser

In [ ]:
# Iniciar simulação
print("Iniciando simulação...")
connector.sim.startSimulation()
connector.sim.step()  # Um passo para estabilizar

# Teste inicial
initial_results = test_laser_in_position(robot_controller, hokuyo_sensor, "Posição Inicial")

### Testes em Diferentes Posições

In [ ]:
print("\n" + "="*60)
print("TESTE EM POSIÇÃO A")
print("Mova o robô para uma nova posição na cena e execute esta célula.")
print("="*60)

wait_for_user_input("Pressione Enter após mover o robô para a posição A...")

# Teste na posição A
position_a_results = test_laser_in_position(robot_controller, hokuyo_sensor, "Posição A")

In [ ]:
print("\n" + "="*60)
print("TESTE EM POSIÇÃO B")
print("Mova o robô para outra posição na cena e execute esta célula.")
print("="*60)

wait_for_user_input("Pressione Enter após mover o robô para a posição B...")

# Teste na posição B
position_b_results = test_laser_in_position(robot_controller, hokuyo_sensor, "Posição B")

In [ ]:
print("\n" + "="*60)
print("TESTE EM POSIÇÃO C")
print("Mova o robô para uma terceira posição na cena e execute esta célula.")
print("="*60)

wait_for_user_input("Pressione Enter após mover o robô para a posição C...")

# Teste na posição C
position_c_results = test_laser_in_position(robot_controller, hokuyo_sensor, "Posição C")

## Exercício 6: Navegação e Mapeamento Incremental

Neste exercício final, utilizamos o script básico de navegação para:

1. Fazer o robô navegar autonomamente pela cena
2. Plotar incrementalmente o caminho executado
3. Combinar todas as leituras de laser ao longo do trajeto
4. Representar tudo em um único gráfico no referencial global

**Algoritmo de navegação:**
- Evitamento de obstáculos baseado em laser
- Comportamento reativo simples
- Coleta contínua de dados para mapeamento

In [ ]:
def autonomous_navigation_with_mapping(duration=30, step_interval=0.1):
    """
    Executa navegação autônoma com mapeamento incremental.

    Args:
        duration: Duração da navegação em segundos
        step_interval: Intervalo entre passos de simulação
    """
    print(f"\n=== Navegação Autônoma por {duration}s ===")

    # Listas para armazenar dados da navegação
    robot_path = []           # Posições do robô
    laser_points_global = []  # Pontos de laser em coordenadas globais
    timestamps = []           # Timestamps para análise

    # Garantir que a simulação está rodando
    if connector.sim.getSimulationState() == 0:
        connector.sim.startSimulation()
        connector.sim.step()

    print("Iniciando navegação autônoma...")
    start_time = time.time()

    try:
        while (sim_time := connector.sim.getSimulationTime()) < duration:
            current_time = time.time() - start_time

            # Obter dados do laser
            laser_data = hokuyo_sensor.getSensorData()

            # Obter pose atual do robô
            robot_pose = robot_controller.get_pose()

            if robot_pose and len(laser_data) > 0:
                position, orientation = robot_pose

                # Armazenar posição do robô
                robot_path.append(position.copy())
                timestamps.append(current_time)

                # Transformar pontos do laser para coordenadas globais
                global_points = transform_laser_to_global(laser_data, robot_pose)

                if len(global_points) > 0:
                    laser_points_global.append(global_points.copy())

                # Algoritmo de navegação (evitamento de obstáculos)
                linear_vel, angular_vel = simple_obstacle_avoidance(laser_data)

                # Aplicar comandos de velocidade
                robot_controller.set_velocity_commands(linear_vel, angular_vel)

                # Log de progresso
                if int(sim_time) % 5 == 0 and sim_time != getattr(autonomous_navigation_with_mapping, 'last_log', -1):
                    print(f"Tempo: {sim_time:.1f}s | Pos: [{position[0]:.2f}, {position[1]:.2f}] | "
                          f"Vel: [v={linear_vel:.2f}, w={np.rad2deg(angular_vel):.1f}°/s] | "
                          f"Laser: {len(laser_data)} pts")
                    autonomous_navigation_with_mapping.last_log = int(sim_time)

            # Avançar simulação
            connector.sim.step()

            # Controle de taxa de atualização
            time.sleep(max(0, step_interval - (time.time() - start_time - current_time)))

    except KeyboardInterrupt:
        print("\nNavegação interrompida pelo usuário")

    finally:
        # Parar o robô
        robot_controller.stop()
        print("Robô parado.")

    print(f"\nNavegação concluída:")
    print(f"- Pontos do trajeto: {len(robot_path)}")
    print(f"- Scans de laser: {len(laser_points_global)}")

    return robot_path, laser_points_global, timestamps

print("Função de navegação autônoma definida.")

### Execução da Navegação Autônoma

In [ ]:
print("🚀 Preparando para navegação autônoma...")
print("O robô irá navegar evitando obstáculos por 25 segundos.")
print("Durante a navegação, os dados de trajetória e laser serão coletados.")

wait_for_user_input("Pressione Enter para iniciar a navegação autônoma...")

# Executar navegação
path_data, laser_data_accumulated, time_stamps = autonomous_navigation_with_mapping(duration=25)

### Visualização dos Resultados

In [ ]:
def create_comprehensive_map(robot_path, laser_points_list, timestamps):
    """
    Cria um mapa abrangente mostrando trajetória e dados de laser.
    """
    print("Criando mapa abrangente...")

    # Preparar dados
    if len(robot_path) == 0:
        print("❌ Nenhum dado de trajetória disponível")
        return

    # Combinar todos os pontos de laser
    all_laser_points = []
    for points in laser_points_list:
        if len(points) > 0:
            all_laser_points.extend(points)

    print(f"Dados processados:")
    print(f"- Pontos de trajetória: {len(robot_path)}")
    print(f"- Pontos de laser totais: {len(all_laser_points)}")

    # Criar figura com subplots
    fig = plt.figure(figsize=(15, 10))

    # Plot principal: Mapa completo
    ax1 = plt.subplot(2, 2, (1, 2))

    # Plotar pontos de laser
    if all_laser_points:
        laser_array = np.array(all_laser_points)
        ax1.scatter(laser_array[:, 0], laser_array[:, 1], c='red', s=0.5,
                   alpha=0.6, label=f'Pontos Laser ({len(all_laser_points)})')

    # Plotar trajetória do robô
    if len(robot_path) > 1:
        path_array = np.array(robot_path)
        ax1.plot(path_array[:, 0], path_array[:, 1], 'b-', linewidth=2,
                label='Trajetória do Robô', alpha=0.8)

        # Marcar início e fim
        ax1.plot(path_array[0, 0], path_array[0, 1], 'go', markersize=10,
                label='Início', markeredgecolor='black')
        ax1.plot(path_array[-1, 0], path_array[-1, 1], 'ro', markersize=10,
                label='Fim', markeredgecolor='black')

        # Adicionar setas de direção a cada 5 pontos
        for i in range(0, len(path_array)-1, max(1, len(path_array)//10)):
            if i < len(path_array)-1:
                dx = path_array[i+1, 0] - path_array[i, 0]
                dy = path_array[i+1, 1] - path_array[i, 1]
                ax1.arrow(path_array[i, 0], path_array[i, 1], dx*0.3, dy*0.3,
                         head_width=0.05, head_length=0.05, fc='blue', ec='blue', alpha=0.7)

    ax1.set_xlabel('X (metros)')
    ax1.set_ylabel('Y (metros)')
    ax1.set_title('Mapa Completo: Navegação e Mapeamento com Laser')
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    ax1.axis('equal')

    # Plot 2: Trajetória X vs Tempo
    ax2 = plt.subplot(2, 2, 3)
    if timestamps and robot_path:
        path_array = np.array(robot_path)
        ax2.plot(timestamps, path_array[:, 0], 'b-', label='X')
        ax2.plot(timestamps, path_array[:, 1], 'r-', label='Y')
        ax2.set_xlabel('Tempo (s)')
        ax2.set_ylabel('Posição (m)')
        ax2.set_title('Trajetória vs Tempo')
        ax2.grid(True, alpha=0.3)
        ax2.legend()

    # Plot 3: Estatísticas
    ax3 = plt.subplot(2, 2, 4)
    ax3.axis('off')

    # Calcular estatísticas
    if robot_path:
        path_array = np.array(robot_path)
        total_distance = 0
        for i in range(1, len(path_array)):
            total_distance += np.linalg.norm(path_array[i] - path_array[i-1])

        stats_text = f"""
ESTATÍSTICAS DA NAVEGAÇÃO

Duração total: {timestamps[-1]:.1f} s
Distância percorrida: {total_distance:.2f} m
Velocidade média: {total_distance/timestamps[-1]:.3f} m/s

Pontos de trajetória: {len(robot_path)}
Scans de laser: {len(laser_points_list)}
Pontos de laser totais: {len(all_laser_points)}

Posição inicial: [{path_array[0,0]:.2f}, {path_array[0,1]:.2f}]
Posição final: [{path_array[-1,0]:.2f}, {path_array[-1,1]:.2f}]
        """

        ax3.text(0.1, 0.9, stats_text, transform=ax3.transAxes, fontsize=10,
                verticalalignment='top', fontfamily='monospace',
                bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.8))

    plt.tight_layout()
    plt.show()

# Gerar visualização completa
if 'path_data' in locals() and path_data:
    create_comprehensive_map(path_data, laser_data_accumulated, time_stamps)
else:
    print("❌ Dados de navegação não disponíveis. Execute a navegação primeiro.")

### Análise Detalhada dos Resultados

In [ ]:
def analyze_navigation_performance(robot_path, laser_points_list, timestamps):
    """
    Análise detalhada do desempenho da navegação.
    """
    print("\n=== Análise de Desempenho da Navegação ===")

    if not robot_path or not timestamps:
        print("❌ Dados insuficientes para análise")
        return

    path_array = np.array(robot_path)

    # 1. Análise de movimento
    print("\n1. ANÁLISE DE MOVIMENTO:")

    # Calcular velocidades instantâneas
    velocities = []
    for i in range(1, len(path_array)):
        dt = timestamps[i] - timestamps[i-1]
        if dt > 0:
            dp = np.linalg.norm(path_array[i] - path_array[i-1])
            vel = dp / dt
            velocities.append(vel)

    if velocities:
        print(f"   Velocidade média: {np.mean(velocities):.3f} m/s")
        print(f"   Velocidade máxima: {np.max(velocities):.3f} m/s")
        print(f"   Velocidade mínima: {np.min(velocities):.3f} m/s")
        print(f"   Desvio padrão: {np.std(velocities):.3f} m/s")

    # 2. Análise de cobertura
    print("\n2. ANÁLISE DE COBERTURA:")

    # Área explorada (bounding box)
    min_x, max_x = np.min(path_array[:, 0]), np.max(path_array[:, 0])
    min_y, max_y = np.min(path_array[:, 1]), np.max(path_array[:, 1])

    print(f"   Área explorada (bounding box): {(max_x-min_x):.2f} x {(max_y-min_y):.2f} m²")
    print(f"   Range X: [{min_x:.2f}, {max_x:.2f}] m")
    print(f"   Range Y: [{min_y:.2f}, {max_y:.2f}] m")

    # 3. Análise de sensoriamento
    print("\n3. ANÁLISE DE SENSORIAMENTO:")

    total_laser_points = sum(len(points) for points in laser_points_list)
    avg_points_per_scan = total_laser_points / len(laser_points_list) if laser_points_list else 0

    print(f"   Total de scans: {len(laser_points_list)}")
    print(f"   Pontos totais detectados: {total_laser_points}")
    print(f"   Média de pontos por scan: {avg_points_per_scan:.1f}")

    # 4. Eficiência de exploração
    print("\n4. EFICIÊNCIA DE EXPLORAÇÃO:")

    # Distância total percorrida
    total_distance = 0
    for i in range(1, len(path_array)):
        total_distance += np.linalg.norm(path_array[i] - path_array[i-1])

    # Distância direta início-fim
    direct_distance = np.linalg.norm(path_array[-1] - path_array[0])

    efficiency = direct_distance / total_distance if total_distance > 0 else 0

    print(f"   Distância total percorrida: {total_distance:.2f} m")
    print(f"   Distância direta (início-fim): {direct_distance:.2f} m")
    print(f"   Eficiência de trajetória: {efficiency:.3f} (1.0 = linha reta)")
    print(f"   Taxa de exploração: {total_laser_points/total_distance:.1f} pontos/metro")

# Executar análise
if 'path_data' in locals() and path_data:
    analyze_navigation_performance(path_data, laser_data_accumulated, time_stamps)
else:
    print("Execute a navegação autônoma primeiro para obter dados para análise.")

## Finalização e Limpeza

In [ ]:
# Parar simulação e limpar recursos
print("\n🔧 Finalizando simulação...")

try:
    # Parar o robô
    if 'robot_controller' in locals():
        robot_controller.stop()
        print("✓ Robô parado")

    # Parar simulação
    if connector.sim.getSimulationState() != 0:
        connector.sim.stopSimulation()
        print("✓ Simulação parada")

    print("\n✅ Exercícios 5-6 concluídos com sucesso!")

except Exception as e:
    print(f"⚠️ Erro durante finalização: {e}")

print("\n📊 Resumo dos exercícios realizados:")
print("   ✓ Exercício 5: Integração do sensor laser e transformações")
print("   ✓ Exercício 6: Navegação autônoma com mapeamento incremental")
print("\n🎯 Todos os objetivos do TP1 foram alcançados!")

## Resumo e Conclusões dos Exercícios 5-6

### Resultados Obtidos:

**Exercício 5:**
- ✅ Integração bem-sucedida do sensor laser Hokuyo
- ✅ Definição correta das transformações T_L_R (laser→robô) e T_W_R (robô→mundo)
- ✅ Transformação de dados de laser para referencial global
- ✅ Validação em múltiplas posições do robô

**Exercício 6:**
- ✅ Implementação de navegação autônoma com evitamento de obstáculos
- ✅ Coleta incremental de dados de trajetória e laser
- ✅ Visualização integrada de caminho e mapeamento
- ✅ Análise quantitativa de desempenho da navegação

### Conceitos Técnicos Aplicados:

1. **Transformações Compostas**: Cadeia laser→robô→mundo
2. **Sensoriamento Ativo**: Integração de dados de laser em tempo real
3. **Navegação Reativa**: Algoritmo de evitamento baseado em sensores
4. **Mapeamento Incremental**: Acumulação de dados sensoriais ao longo do tempo
5. **Análise de Desempenho**: Métricas de eficiência e cobertura

### Implementação Robótica:

- **Modularidade**: Classes especializadas para cada componente
- **Tempo Real**: Processamento eficiente para navegação autônoma
- **Robustez**: Tratamento de erros e validações
- **Visualização**: Plots informativos para análise dos resultados
- **Documentação**: Código bem estruturado e comentado

### Aplicações Práticas:

Os conceitos implementados neste TP1 são fundamentais para:
- Robôs de serviço em ambientes internos
- Veículos autônomos para mapeamento
- Sistemas de navegação robótica
- Algoritmos de SLAM (Simultaneous Localization and Mapping)

### Trabalhos Futuros:

Este trabalho estabelece a base para:
- Implementação de algoritmos de SLAM
- Planejamento de caminhos mais sofisticados
- Integração de múltiplos sensores
- Navegação em ambientes dinâmicos